In [ ]:
# # run this cell every time you restart
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))
# !pip install gensim
print("ok")

In [23]:
import numpy as np
from keras.models import Sequential, Model, load_model
from keras.layers import TimeDistributed, RepeatVector, LSTM, Dense, Embedding, Input, concatenate
from keras.preprocessing.text import Tokenizer, one_hot, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from dataset_loader import load_dataset
from collections import Counter
from random import randrange, random
from string import punctuation
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from pickle import load
from copy import deepcopy as dc

In [ ]:
x, y = load_dataset()
with open("fixed_len_dataset.pkl", "rb") as fp:
  x = load(fp)

x, y = zip(*list(filter(lambda x : len(text_to_word_sequence(x[1])) == 60, zip(x,y)))) #Selects only those data points whose summary length is 60 words

all_words = " ".join(x) + " ".join(y)
all_words = text_to_word_sequence(all_words)

print(x[0])
print(y[0])

vocab_size = len(set(all_words)) #56558, 27178 for only summaries with 60 words

print("Size of vocabulary : ", vocab_size)

In [5]:
# tokenized x, y and xy
tok_x = [xi.split() for xi in x]
tok_y = [yi.split()[:60] + max(0, 60 - len(yi.split())) * ["__unk__"] for yi in y]
tok_xy = [xi + yi for xi, yi in zip(tok_x, tok_y)]

# len_tok_x = set([len(xi) for xi in tok_x])
# print(len_tok_x)

# len_tok_y = set([len(yi) for yi in tok_y])
# print(len_tok_y)

print(len(tok_xy[0]))
print(tok_xy[0])

tok_xy.append(5 * ["__unk__"])

# word2vec model
model_w2v = Word2Vec(tok_xy, size = 100, min_count = 5)

# size of vocab
words = list(model_w2v.wv.vocab)
vocab_size = len(words)
print(vocab_size)

# replacing unknown words
tok_x = [[word if word in model_w2v.wv.vocab else "__unk__" for word in xi] for xi in tok_x]
tok_y = [[word if word in model_w2v.wv.vocab else "__unk__" for word in yi] for yi in tok_y]

# x values
x_train_article = np.array([[model_w2v[word] for word in xi] for xi in tok_x])
x_train_summary = np.array([[model_w2v[word] for word in yi] for yi in tok_y])

print(x_train_article.shape)
print(x_train_summary.shape)

del tok_x
del tok_xy

424
['The', 'Daman', 'and', 'Diu', 'administration', 'on', 'Wednesday', 'withdrew', 'a', 'circular', 'that', 'asked', 'women', 'staff', 'to', 'tie', 'rakhis', 'on', 'male', 'colleagues', 'after', 'the', 'order', 'triggered', 'a', 'backlash', 'from', 'employees', 'and', 'was', 'ripped', 'apart', 'on', 'social', 'media.The', 'union', 'territory?s', 'administration', 'was', 'forced', 'to', 'retreat', 'within', '24', 'hours', 'of', 'issuing', 'the', 'circular', 'that', 'made', 'it', 'compulsory', 'for', 'its', 'staff', 'to', 'celebrate', 'Rakshabandhan', 'at', 'workplace.?It', 'has', 'been', 'decided', 'to', 'celebrate', 'the', 'festival', 'of', 'Rakshabandhan', 'on', 'August', '7.', 'In', 'this', 'connection,', 'all', 'offices/', 'departments', 'shall', 'remain', 'open', 'and', 'celebrate', 'the', 'festival', 'collectively', 'at', 'a', 'suitable', 'time', 'wherein', 'all', 'the', 'lady', 'staff', 'shall', 'tie', 'rakhis', 'to', 'their', 'colleagues,?', 'the', 'order,', 'issued', 'on', 'Au

C:\Users\Varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\Varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(4396,)
(4396, 60, 100)


In [ ]:
print(words[:10])

# integer encode
label_encoder = LabelEncoder()
label_encoder.fit(words)

print(label_encoder.transform(["world"]))

# one-hot encode
label_binarizer = LabelBinarizer()
label_binarizer.fit(range(vocab_size))

y_train = np.array([label_binarizer.transform(label_encoder.transform(yi)) for yi in tok_y])
print(y_train.shape)

# required shape -> 1112, 60, vocab_size

# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)
#print(words[inverted])

In [ ]:
# Source:  Alternate 3 from https://machinelearningmastery.com/encoder-decoder-models-text-summarization-keras/
epochs = 16
batch_size = 32

src_txt_length = 100
sum_txt_length = 60
word_vec_length = 100

# article input model
inputs1 = Input(shape=(src_txt_length, word_vec_length))
# article1 = Embedding(vocab_size, 128)(inputs1)
article2 = LSTM(128)(inputs1)
article3 = RepeatVector(sum_txt_length)(article2)

# summary input model
inputs2 = Input(shape=(sum_txt_length, word_vec_length))
# summ1 = Embedding(vocab_size, 128)(inputs2)

# decoder model
decoder1 = concatenate([article3, inputs2])
decoder2 = LSTM(128, return_sequences = True)(decoder1)                           # made changes here
outputs = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder2)      # made changes here

# tie it together [article, summary] [word]
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['acc'])
model.fit([x_train_article, x_train_summary], y_train, epochs = epochs, batch_size = batch_size)

In [3]:
from math import log
from pickle import load
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
from random import randrange
from word_score import get_word_score
from sentence_score import extractive_summarizer 
import string

model.save('my-model.h5')

if __name__ == "__main__" :
	with open('dataset.pkl', 'rb') as fp:
		article_word_list, summary_word_list = load(fp)

	word_score_dictionary = get_word_score(article_word_list, summary_word_list)
	x, y = load_dataset()
	fixed_x = []
	for i in range(len(x)):
		x[i].maketrans('','', string.punctuation)
		if(len(x[i].split()) > 150):
			ans = " ".join(e for e in x[i].split()[:100])
			fixed_x.append(ans)
		else:
			try :
				summary = extractive_summarizer(x[i], word_score_dictionary, sentence_count = 6)
			except ZeroDivisionError :
				continue

			z = summary.split()
			if(len(z)>100):
				ans = " ".join(e for e in z[:100])
				fixed_x.append(ans)
			else:
				pre_len = len(z)
				for i in range(100 - pre_len):
					z.append(z[randrange(pre_len)])
				ans = " ".join(e for e in z)	
				fixed_x.append(ans)

Va:  4341
Vs:  4396
Na:  4396
Ns:  4396


In [2]:
#model.save("my_model.h5")
model = load_model("my_model.h5")

In [ ]:
#!ls
#files.download("my_model.h5")

In [ ]:
# def preprocess_input(article) :
#   article = article.maketrans('', '', string.punctuation)
#   article = " ".join(article.split()[:100])
#   vector = []
#   for word in article.split() :
#     vector.append(model_w2v[word])
#   return vector
# # 	if(len(x[i].split()) < 150):
# #     ans = " ".join(e for e in article.split()[:100])
# # 	else:
# # 		 article = extractive_summarizer(article, word_score_dictionary, sentence_count = 6)


In [17]:
some_article = fixed_x[3]
print(some_article)

Lashkar-e-Taiba's Kashmir commander Abu Dujana was killed in an encounter in a village in Pulwama district of Jammu and Kashmir earlier this week. Dujana, who had managed to give the security forces a slip several times in the past, carried a bounty of Rs 15 lakh on his head.Reports say that Dujana had come to meet his wife when he was trapped inside a house in Hakripora village. Security officials involved in the encounter tried their best to convince Dujana to surrender but he refused, reports say.According to reports, Dujana rejected call for surrender from an Army officer. The Army


In [32]:
def normalize_text(text, lim = 100) :
    text = text.lower()
    text = text.replace("'", "") #wouldn't -> wouldnt
    for char in list(punctuation) : #remove all punctuation
        if char != "-" and char != "-":
            text = text.replace(char, " ")
    #text = " ".join(filter(lambda x : x not in stopwords , text.split())) #remove stopwords
    #text = " ".join(filter(lambda x : x.isalpha(), text.split())) #remove numbers
    #text = " ".join([porter.stem(x) for x in text.split()]) #stemming
    text = " ".join(text.split()) #ensure words delimited by single space only
    return " ".join(text.split()[:lim])

inp = []
some_article = normalize_text(some_article)
print(some_article)
for word in some_article.split() :
    if word not in model_w2v :
        inp.append(np.array([random() for x in range(100)]))
    else :
        inp.append(model_w2v[word])
inp = np.array(inp)
# summary = []
# print(len(model_w2v.vocab_))

lashkar-e-taibas kashmir commander abu dujana was killed in an encounter in a village in pulwama district of jammu and kashmir earlier this week dujana who had managed to give the security forces a slip several times in the past carried a bounty of rs 15 lakh on his head reports say that dujana had come to meet his wife when he was trapped inside a house in hakripora village security officials involved in the encounter tried their best to convince dujana to surrender but he refused reports say according to reports dujana rejected call for surrender from an army officer


C:\Users\Varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Users\Varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [52]:
arr = np.array([0 for x in range(100)])
summary = np.array([dc(arr) for x in range(60)])
print(type(summary), summary.shape, type(inp), inp.shape)

<class 'numpy.ndarray'> (60, 100) <class 'numpy.ndarray'> (100, 100)


In [53]:
k = model.predict([np.array([inp]), np.array([summary])])
print(k)
print(k.shape)

[[[  1.30756554e-04   9.88543325e-05   1.37591109e-04 ...,   1.10135261e-04
     1.03668528e-04   1.71137835e-05]
  [  1.03160237e-04   6.57361816e-05   1.03010214e-04 ...,   7.73525462e-05
     6.88751607e-05   5.84975851e-06]
  [  1.03154489e-04   6.20565843e-05   9.96522940e-05 ...,   7.28034429e-05
     6.42519080e-05   4.64172854e-06]
  ..., 
  [  1.14317081e-04   5.54842845e-05   1.01215497e-04 ...,   6.70053196e-05
     5.99363739e-05   3.26151803e-06]
  [  1.14365066e-04   5.55173974e-05   1.01269805e-04 ...,   6.70268273e-05
     5.99073501e-05   3.25785140e-06]
  [  1.14412709e-04   5.55506485e-05   1.01323356e-04 ...,   6.70485897e-05
     5.98794941e-05   3.25429414e-06]]]
(1, 60, 4497)


In [63]:
def create_summary_vec(summary_so_far = []) :
    unk = model_w2v["__unk__"]
    return np.array([unk for x in range(60)])

current_summary= create_summary_vec()
summary_text = []
for i in range(60) :
    k = model.predict([np.array([inp]), np.array([current_summary])])
    inverted = label_encoder.inverse_transform([np.argmax(i) for i in k[0]])
    try :
        current_summary[i] = model_w2v[inverted[i]]
        summary_text.append(inverted[i])
    except :
        current_summary[i] = model_w2v["__unk__"]
print(summary_text)


C:\Users\Varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\Varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


['$100,000', '$100,000', '$2', '$2', '$2', '$2', '$2', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Budget', 'Davis', 'Davis', 'Davis', 'Davis', 'Davis', 'Davis', 'Hillary', 'Hillary', 'Hillary', 'Hillary', 'Hillary', 'Hillary', 'Hillary', 'Hillary', 'Hillary', 'Hillary', 'Hillary', 'Hillary', 'Hillary', 'Hillary', 'Has', 'Has', 'Has', 'Has', 'Has', 'Has', 'Has']


In [50]:
inverted = label_encoder.inverse_transform(current_summary)
print(inverted)

C:\Users\Varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


ValueError: Error when checking : expected input_2 to have shape (60, 100) but got array with shape (1, 100)